# Dependencies

În acest moment, ne-am creat toate toask-urile de care avem nevoie pentru a rula acest proiect. Dacă ne uităm în Airflow (în UI) la acest DAG, în secțiunea de Graph View putem observa faptul că avem toate task-urile prezente

<img src="../../ss/airflow-section-03/section-03-ss-13.png">

Se pot observa task-urile respective, dar după cum se mai poate observa, acestea nu par a fi în ordine. Primul task, cel mai de sus este acela în care se creează tabelul în HDFS pentru a scrie datele, însă acest lucru nu putem să îl facem dacă API-ul nu este disponibil și nu putem descărca datele. Prin urmare, trebuie să stabilim ordinea în care o să se ruleze aceste task-uri individuale. Acest procedeu poartă denumirea de "Tasks dependencies"

O să luăm primele două task-uri pe care le-am creat, cel prin care verificăm API-ul și fișierul local. O să îi spunem lui Airflow ca prima dată să ruleze task-ul "is_forex_rates_available", iar după ce termină de rulat acest task, să ruleze task-ul "is_forex_file_available". Ca să facem asta, prima dată preluăm denumirea task-ului și o copiem la final după definirea tuturor task-urilor.


... python code

    is_forex_rates_available

Acest lucru o să ruleze task-ul respectiv. Ca să îi spunem să aștepte terminarea acestui task și după să înceapă urătorul task, o să ne folosim de semnul ">>". Acest semn îi spune lui Airflow să ruleze primul task, să aștepte să fie rulat și abia după să treacă la următorul

... python code

    is_forex_rates_available >> is_forex_file_available

O să introducem acest cod în codul de DAG și o să verificăm în Airflow în UI cum arată acuma graficul respectiv.

<img src="../../ss/airflow-section-03/section-03-ss-14.png">

După cum se poate observa acuma, între task-ul de "is_forex_rates_available" și "is_forex_file_available" există o săgeată îndreptată dinspe "is_forex_rates_available" către "is_forex_file_available" prin care se arată faptul că prima dată se rulează task-ul 'is_forex_rates_available' și după task-ul "is_forex_file_available". Așa trebuie să procedăm cu toate task-urile respective ca să le punem în ordinea în care trebuie

... python code

    is_forex_rates_available >> is_forex_file_available >> download_forex_rates >> save_forex_rates_to_hdfs >> create_forex_rates_table_hdfs >> process_forex_rates_spark

Linia de mai sus este una destul de mare, iar uneori este greu să urmărim o linie atâta de mare. Pentru asta putem să împărțim aceată linie în mai multe etape

... python code

    # check with sensors
    is_forex_rates_available >> is_forex_file_available

    # download the data
    is_forex_file_available >> download_forex_data

    # HDFS processing
    download_forex_data >> save_forex_rates_to_hdfs >> create_forex_rates_table_hdfs

    # Spark processing
    create_forex_rates_table_hdfs >> process_forex_rates_spark

După ce am setat toate aceste dependințe pentru task-uri, putem să vedem cum arată acestea în Airflow

<img src="../../ss/airflow-section-03/section-03-ss-15.png">

Din moment ce vedem că toate aceste task-uri sunt în ordinea corectă, putem să pormin acest pipeline

<img src="../../ss/airflow-section-03/section-03-ss-16.png">

După cum se poate observa, fiecare task a rulat cu succes (acel pătrat verde înseamnă că s-a rulat cu succes task-ul respectiv). Am creat primul pipeline de date utilizând Airflow, Spark și HDFS